In [1]:
import time
from datetime import datetime

from lerobot.cameras.opencv.configuration_opencv import OpenCVCameraConfig
from lerobot.datasets import lerobot_dataset
from lerobot.datasets.lerobot_dataset import LeRobotDataset
from lerobot.datasets.utils import hw_to_dataset_features
from lerobot.processor.factory import make_default_processors
from lerobot.robots.so101_follower import SO101Follower, SO101FollowerConfig
from lerobot.scripts.lerobot_record import record_loop
from lerobot.teleoperators.so101_leader import SO101Leader, SO101LeaderConfig
from lerobot.utils.control_utils import init_keyboard_listener
from lerobot.utils.utils import log_say
from lerobot.utils.visualization_utils import init_rerun

# Monkey Patch: Disable expensive image stats computation during recording
_original_compute = lerobot_dataset.compute_episode_stats


def fast_compute_stats(episode_data, features, *args, **kwargs):
    # Filter out image/video features so we don't read them from disk
    non_image_features = {
        k: v for k, v in features.items() if v["dtype"] not in ["image", "video"]
    }
    # Only compute stats for vectors (fast)
    try:
        # Filter episode data to only include non-image keys to prevent errors in compute_episode_stats
        filtered_episode_data = {
            k: v for k, v in episode_data.items() if k in non_image_features
        }
        return _original_compute(
            filtered_episode_data, non_image_features, *args, **kwargs
        )
    except Exception as e:
        print(f"Error computing stats: {e}")
        return {}


lerobot_dataset.compute_episode_stats = fast_compute_stats

FPS = 30
NUM_EPISODES = 2
EPISODE_TIME_SEC = 20
RESET_TIME_SEC = 2
TASK_DESCRIPTION = "Place box on plate"

camera_config = {
    "front": OpenCVCameraConfig(
        index_or_path=6, width=640, height=480, fps=30, fourcc="MJPG"
    ),
    "top": OpenCVCameraConfig(
        index_or_path=4, width=1280, height=720, fps=30, fourcc="MJPG"
    ),
}

robot_config = SO101FollowerConfig(
    port="/dev/ttyACM0", id="my_awesome_follower_arm", cameras=camera_config
)

teleop_config = SO101LeaderConfig(
    port="/dev/ttyACM1",
    id="my_awesome_leader_arm",
)



print("Starting record dataset...")

robot = SO101Follower(robot_config)
teleop = SO101Leader(teleop_config)

# Configure the dataset features
action_features = hw_to_dataset_features(robot.action_features, "action")
obs_features = hw_to_dataset_features(robot.observation_features, "observation")
dataset_features = {**action_features, **obs_features}

# Create the dataset
dataset = LeRobotDataset.create(
    repo_id=f"felix-newman/test-robot-dataset-{datetime.now().strftime('%Y%m%d%H%M%S')}",
    fps=FPS,
    features=dataset_features,
    robot_type=robot.name,
    use_videos=True,
    image_writer_threads=4,
    batch_encoding_size=2,
)

# Standard Processors. Take the to_transitation and to output functions and apply
# a noop step in between
teleop_action_processor, robot_action_processor, robot_observation_processor = (
    make_default_processors()
)

# Initialize the keyboard listener and rerun visualization
_, events = init_keyboard_listener()
init_rerun(session_name="recording")

robot.connect(calibrate=False)
teleop.connect(calibrate=False)

episode_idx = 0
while episode_idx < NUM_EPISODES and not events["stop_recording"]:
    try:
        log_say(f"Recording episode {episode_idx + 1} of {NUM_EPISODES}")

        record_loop(
            robot=robot,
            events=events,
            fps=FPS,
            teleop_action_processor=teleop_action_processor,
            robot_action_processor=robot_action_processor,
            robot_observation_processor=robot_observation_processor,
            teleop=teleop,
            dataset=dataset,
            control_time_s=EPISODE_TIME_SEC,
            single_task=TASK_DESCRIPTION,
            display_data=True,
        )

        # Reset the environment if not stopping or re-recording
        if not events["stop_recording"] and (
            episode_idx < NUM_EPISODES - 1 or events["rerecord_episode"]
        ):
            log_say("Reset the environment")
            print("Reset the environment")
            start = time.perf_counter()
            record_loop(
                robot=robot,
                events=events,
                fps=FPS,
                teleop_action_processor=teleop_action_processor,
                robot_action_processor=robot_action_processor,
                robot_observation_processor=robot_observation_processor,
                teleop=teleop,
                control_time_s=RESET_TIME_SEC,
                single_task=TASK_DESCRIPTION,
                display_data=True,
            )
            end = time.perf_counter()
            print(f"Reset the environment took {end - start:.3f} seconds")

        if events["rerecord_episode"]:
            log_say("Re-recording episode")
            events["rerecord_episode"] = False
            events["exit_early"] = False
            dataset.clear_episode_buffer()
            continue

        print("Saving episode")
        start = time.perf_counter()
        dataset.save_episode()
        end = time.perf_counter()
        print(f"Save episode took {end - start:.3f} seconds")
        episode_idx += 1

        if episode_idx >= NUM_EPISODES:
            break
    except Exception as e:
        print(f"Error recording episode {episode_idx}: {e}")
        log_say(
            f"Error recording episode {episode_idx}. Saving dataset and exiting..."
        )
        break

robot.disconnect()
teleop.disconnect()

Starting record dataset...


[2025-11-27T10:30:43Z INFO  re_grpc_server] Listening for gRPC connections on 0.0.0.0:9876. Connect by running `rerun --connect rerun+http://127.0.0.1:9876/proxy`
[2025-11-27T10:30:43Z INFO  winit::platform_impl::linux::x11::window] Guessed window scale factor: 1
[2025-11-27T10:30:43Z WARN  wgpu_hal::vulkan::instance] Unable to find extension: VK_EXT_physical_device_drm
[2025-11-27T10:30:43Z INFO  egui_wgpu] There were 4 available wgpu adapters: {backend: Vulkan, device_type: IntegratedGpu, name: "Intel(R) Graphics (RPL-P)", driver: "Intel open-source Mesa driver", driver_info: "Mesa 24.2.8-1ubuntu1~24.04.1", vendor: Intel (0x8086), device: 0xA7A0}, {backend: Vulkan, device_type: DiscreteGpu, name: "NVIDIA GeForce RTX 4060 Laptop GPU", driver: "NVIDIA", driver_info: "570.195.03", vendor: NVIDIA (0x10DE), device: 0x28A0}, {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 19.1.1, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 24.2.8-1ubuntu1~24.04.1 (LLVM 19.1.1)", vendor: Me

Waiting for -0.006 seconds
Waiting for 0.024 seconds
Waiting for 0.026 seconds
Waiting for 0.027 seconds
Waiting for 0.028 seconds
Waiting for 0.028 seconds
Waiting for 0.027 seconds


Corrupt JPEG data: 1 extraneous bytes before marker 0xd4
Corrupt JPEG data: 6 extraneous bytes before marker 0xd6
Corrupt JPEG data: 1 extraneous bytes before marker 0xd3
Corrupt JPEG data: 2 extraneous bytes before marker 0xd7
Corrupt JPEG data: 5 extraneous bytes before marker 0xd2
Corrupt JPEG data: 3 extraneous bytes before marker 0xd6
Corrupt JPEG data: 2 extraneous bytes before marker 0xd1


Waiting for 0.028 seconds
Waiting for 0.028 seconds
Waiting for 0.028 seconds
Waiting for 0.028 seconds
Waiting for 0.027 seconds
Waiting for 0.028 seconds
Waiting for 0.027 seconds
Waiting for 0.027 seconds
Waiting for 0.028 seconds
Waiting for 0.027 seconds
Waiting for 0.026 seconds
Waiting for 0.026 seconds
Waiting for 0.027 seconds


...

Corrupt JPEG data: 5 extraneous bytes before marker 0xd5
Corrupt JPEG data: 11 extraneous bytes before marker 0xd3
Corrupt JPEG data: 3 extraneous bytes before marker 0xd2


Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Corrupt JPEG data: 2 extraneous bytes before marker 0xd4
Corrupt JPEG data: 2 extraneous bytes before marker 0xd1


Error recording episode 1: 'NoneType' object is not subscriptable


Corrupt JPEG data: 1 extraneous bytes before marker 0xd5


In [ ]:
vars(dataset.meta)['stats']

{'action': {'min': array([  1.2048193, -99.40426  ,  99.909256 ,  51.288887 ,  -0.6246747,
           0.6677796], dtype=float32),
  'max': array([  1.2048193, -99.40426  ,  99.909256 ,  51.288887 ,  -0.6246747,
           0.6677796], dtype=float32),
  'mean': array([  1.204819  , -99.40428665,  99.90922059,  51.28888597,
          -0.62467452,   0.66777951]),
  'std': array([1.30456991e-03, 3.61520604e-06, 1.15136074e-01, 1.44608242e-05,
         5.62505015e-04, 6.42490359e-04]),
  'count': array([94]),
  'q01': array([  1.20481932, -99.40425873,  99.90925598,  51.28888702,
          -0.62467468,   0.66777962]),
  'q10': array([  1.20481932, -99.40425873,  99.90925598,  51.28888702,
          -0.62467468,   0.66777962]),
  'q50': array([  1.20481932, -99.40425873,  99.90925598,  51.28888702,
          -0.62467468,   0.66777962]),
  'q90': array([  1.20481932, -99.40425873,  99.90925598,  51.28888702,
          -0.62467468,   0.66777962]),
  'q99': array([  1.20481932, -99.40425873,  99

Right arrow key pressed. Exiting loop...
Right arrow key pressed. Exiting loop...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Left arrow key pressed. Exiting loop and rerecord the last episode...
Right ar